Copyright **`(c)`** 2021 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see 'LICENCE.md' for details.

# Connect 4

In [153]:
from collections import Counter
import numpy as np
from IPython.display import clear_output

import os

In [154]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"

## Basic Functions

In [155]:
clear = lambda: os.system('cls')

def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )

def print_board(board):
    corr = board.copy()
    corr = np.array(board).T[::-1].reshape(NUM_COLUMNS*COLUMN_HEIGHT)
    corr = ["X" if i==1 else i for i in corr]
    corr = ["O" if i==-1 else i for i in corr]
    corr = [" " if i==0 else i for i in corr]
    corr = np.array(corr).reshape((COLUMN_HEIGHT,NUM_COLUMNS))
    
            
    #for i in range(corr.shape[0]):
        #corr[i] = ["X" if j==1 else j for j in corr[i]]
        #corr[i] = ["O" if j==-1 else j for j in corr[i]]
        #corr[i] = [' ' if j==0 else j for j in corr[i]]

    clear_output(wait=True)
    print(np.array2string(corr, separator=' '))

## Montecarlo Evaluation

In [168]:
def _mc(board, player):
    p = -player
    while valid_moves(board):
        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)
        if four_in_a_row(board, p):
            return p
    return 0


def montecarlo(board, player):
    montecarlo_samples = 500
    cnt = Counter(_mc(np.copy(board), player) for _ in range(montecarlo_samples))
    return (cnt[1] - cnt[-1]) / montecarlo_samples


def eval_board(board, player):
    if four_in_a_row(board, 1):
        # Alice won
        return 1
    elif four_in_a_row(board, -1):
        # Bob won
        return -1
    else:
        # Not terminal, let's simulate...
        return montecarlo(board, player)

In [170]:
def minmaxmc(board, player, level): #player is player in this turn
    
    p1 = four_in_a_row(board,player)
    if p1:
        return None, player

    possible = valid_moves(board)
    if not possible:
        return None, 0   

    if level==1:
        return None, player*montecarlo(board,player)

    evaluations = list()

    for ply in possible:
        new_board = board.copy()
        play(new_board,ply,player)
        _, val = minmaxmc(new_board, -player, level+1)
        evaluations.append((ply, -val))
        #print(new_board)
    return max(evaluations, key=lambda k: k[1])

In [ ]:
#BASIC (WORKING) SOLUTION WITH MONTECARLO
board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
p1 = False
p2 = False
player = 1
while p1==False and p2==False:
    evaluations = list()
    possible = valid_moves(board)
    for ply in possible:
        new_board = board.copy()
        play(new_board,ply,player)
        val = montecarlo(new_board, player)
        evaluations.append((ply, val))
    pos, val = max(evaluations, key=lambda k: k[1])
    play(board, pos, player)
    player = -player
    p1 = four_in_a_row(board,1)
    p2 = four_in_a_row(board,-1)
    print(board)
p1 = four_in_a_row(board,1)
p2 = four_in_a_row(board,-1)
if p1==True:
    print("p1 won")
elif p2==True:
    print("p2 won")
else:
    print("tie")



## TRY also with depth 3 and 25 samples
## depth 2, 100 samples, 13m 31s, 43s for initial move) -> tie
## depth 1, 200 samples, 18s each move, 3m 31s
## depth 1, 500 samples, 5m 22s

In [172]:
#WORKING SOLUTION WITH MINMAX2+MONTECARLO DEPTH n?

board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
print_board(board)
p = False
player = 1
while p==False:
    ply, val = minmaxmc(board,player,0)
    play(board, ply, player)
    p = four_in_a_row(board,player)
    player = -player
    clear()
    print_board(board)
p1 = four_in_a_row(board,1)
p2 = four_in_a_row(board,-1)
if p1==True:
    print("p1 won")
elif p2==True:
    print("p2 won")
else:
    print("tie")


[[' ' ' ' 'X' 'O' ' ' ' ' ' ']
 [' ' 'O' 'X' 'X' ' ' ' ' ' ']
 [' ' 'X' 'O' 'X' 'X' ' ' ' ']
 [' ' 'O' 'O' 'X' 'O' 'X' ' ']
 [' ' 'O' 'X' 'O' 'O' 'O' ' ']
 ['X' 'O' 'X' 'X' 'O' 'X' ' ']]
p1 won


## WITH AB PRUNING